In [1]:
import numpy as np
import pandas as pd

from data_processing_functions import get_train_data, get_test_data, get_train_and_test_data, convert_str_variable

## 1. Read Data

In [ ]:
#Error - ?
data_train, dest_test = get_train_and_test_data('./data', amount_of_days=3, wind_border=8, convert_str_variable_flag=True)
x_data_train, y_data_wind_train, y_data_temperature_train = data_train
x_data_test, y_data_wind_test, y_data_temperature_test = data_train

In [2]:
data_train, dest_test = get_train_and_test_data('./data', amount_of_days=3, wind_border=8, convert_str_variable_flag=False)
x_data_train, y_data_wind_train, y_data_temperature_train = data_train
x_data_test, y_data_wind_test, y_data_temperature_test = data_train

In [3]:
x_data_train, columns, dicts = convert_str_variable(x_data_train, 3)
x_data_test = convert_str_variable(x_data_test, 3, columns=columns, dicts=dicts)[0]

## 2. Wind classification

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [5]:
clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1)

In [6]:
clf.fit(x_data_train, y_data_wind_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(100, 20), random_state=1)

In [7]:
accuracy_score(clf.predict(x_data_test), y_data_wind_test)

0.8553809112796984

## 3. Temperature prediction

In [9]:
from sklearn.neural_network import MLPRegressor

In [10]:
regr = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1)

In [11]:
regr.fit(x_data_train, y_data_temperature_train)

C:\Users\Patryk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPRegressor(alpha=1e-05, hidden_layer_sizes=(100, 20), random_state=1,
             solver='lbfgs')

In [12]:
err = abs(regr.predict(x_data_test) - y_data_temperature_test)
MAE = round(np.mean(err),2)
RMSE = round(np.sqrt(((err)**2).mean()),2)
print("MAE:", MAE, "RMSE:", RMSE)

MAE: 5.05 RMSE: 6.92
